# Imports, settings and paths

In [30]:
import pandas as pd
import os
from copy import deepcopy
from random import randint

In [152]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import SingleLetterAlphabet
from tqdm import tqdm_notebook as tqdm
from ete3 import NCBITaxa

In [3]:
# path_ncbi = "/home/ubuntu/Disks/HDD1000/NCBI/20190704/refseq"
path_ncbi_csv = "/home/ubuntu/Disks/HDD1000/NCBI/ncbi.csv"
path_ncbi_pd = "/home/ubuntu/Disks/HDD1000/NCBI/ncbi.pd"

In [4]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 500)

# SyntRead Class

In [174]:
class SyntRead:
    count = 0
    
    def __init__(self, record_id, taxon, genome, what, s_kingdom, path_complete, description):  # folder, file,
        self.object_id    = SyntRead.counter()
        self.record_id    = record_id
        self.taxon        = taxon
        self.genome       = genome
        self.gen_chr_plas = what
        self.s_kingdom    = s_kingdom
        self.path         = path_complete  # os.path.join(folder, file)
        self.description  = description
        self.description_original = description
        
        self.seq          = None
        self.seq_len      = None
        self.start        = None
        self.stop         = None
        self.read_len     = None
        self.read         = None 
        
    def counter():
        SyntRead.count += 1
        return SyntRead.count - 1
        
    def synthetize_read(self, seq=""):
        if seq == "":
            records = list(SeqIO.parse(self.path, "fasta"))
            for r in records:
                if r.id == self.record_id:
                    seq = str(r.seq)
                    break
                else:
                    raise FileNotFoundError("File error, couldn't find the right record")
            self.seq_len = len(seq)
            
        self.read_len  = randint(1000, 10000)
        self.start     = randint(0, self.seq_len - self.read_len)
        self.stop      = self.start + self.read_len
        self.read      = seq[self.start:self.stop]
        self.update_description()
        
    def deep_random_copy(self, n):
        seq = self.fetch_seq()
        l = []
        for i in range(n):
            new_item = deepcopy(self)
            new_item.synthetize_read(seq)
            l.append(new_item)
        return l
    
    def update_description(self):
        self.description = self.description_original + f", synthetic read from this genome (taxid={self.taxon}), "\
                                                       f"from nucleotide {self.start} to {self.stop}(len={self.read_len})"
        
    def fetch_seq(self):
        records = list(SeqIO.parse(self.path, "fasta"))
        for r in records:
            if r.id == self.record_id:
                seq = str(r.seq)
                break
            else:
                raise FileNotFoundError("File error, couldn't find the right record")
        return seq
    
    def __repr__(self):
        return f"Synthetic read from {self.genome}, len={self.read_len}, [{self.start}:{self.stop}]"
        
    def to_fastq(list_sr, path_fastq):
        records = []
        for sr in tqdm(list_sr):
            records.append(SeqRecord(Seq(sr.read, SingleLetterAlphabet), 
                                     id=sr.record_id, name=sr.genome, description=sr.description))
        with open(path_fastq, "w") as f:
            SeqIO.write(records, f, "fasta")

# Create List of SyntReads

In [6]:
df = pd.read_pickle(path_ncbi_pd)

In [7]:
df.shape

(578756, 25)

In [8]:
df[(df.name_superkingdom == "Bacteria") | (df.name_superkingdom == "Archea")].shape

(26647, 25)

In [9]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()

ncbi.get_fuzzy_name_translation("Clostridioides difficile")

ncbi.get_name_translator(["Clostridioides difficile"])

In [140]:
df.sample(2)

,taxon_id,taxon_name,rank,type,length,record_id,record_description,path_folder,path_file,id_superkingdom,name_superkingdom,id_kingdom,name_kingdom,id_phylum,name_phylum,id_class,name_class,id_order,name_order,id_family,name_family,id_genus,name_genus,id_species,name_species
17887,1618022,Nostoc sp. 'Lobaria pulmonaria (5183) cyanobiont',species,plasmid,121770,NZ_CP026693.1,NZ_CP026693.1 Nostoc sp. 'Lobaria pulmonaria (...,/home/ubuntu/Disks/HDD1000/NCBI/20190704/refseq/,bacteria/GCF_002949795.1/GCF_002949795.1_ASM29...,2,Bacteria,1117,Cyanobacteria,1161,Nostocales,1162,Nostocaceae,1177,Nostoc,1618022,Nostoc sp. 'Lobaria pulmonaria (5183) cyanobiont',0,NaN,0,NaN
451580,4686,Asparagus officinalis,species,undefined,562,NW_017980223.1,NW_017980223.1 Asparagus officinalis unplaced ...,/home/ubuntu/Disks/HDD1000/NCBI/20190704/refseq/,plant/GCF_001876935.1/GCF_001876935.1_Aspof.V1...,2759,Eukaryota,4447,Liliopsida,4685,Asparagus,4686,Asparagus officinalis,33090,Viridiplantae,35493,Streptophyta,40552,Asparagaceae,73496,Asparagales


In [141]:
df[(df.type == "complete genome") 
   & ((df.name_superkingdom == "Bacteria") 
      | (df.name_superkingdom == "Archea"))].sample(2)  #[["taxon_id", "taxon_name", "rank"]]

,taxon_id,taxon_name,rank,type,length,record_id,record_description,path_folder,path_file,id_superkingdom,name_superkingdom,id_kingdom,name_kingdom,id_phylum,name_phylum,id_class,name_class,id_order,name_order,id_family,name_family,id_genus,name_genus,id_species,name_species
6161,1382924,Helicobacter pylori oki422,no rank,complete genome,1634852,NZ_CP006824.1,"NZ_CP006824.1 Helicobacter pylori oki422, comp...",/home/ubuntu/Disks/HDD1000/NCBI/20190704/refseq/,bacteria/GCF_000600165.1/GCF_000600165.1_ASM60...,2,Bacteria,209,Helicobacter,210,Helicobacter pylori,1224,Proteobacteria,29547,Epsilonproteobacteria,72293,Helicobacteraceae,213849,Campylobacterales,0,NaN
12540,197,Campylobacter jejuni,species,complete genome,1629484,NZ_CP007189.1,NZ_CP007189.1 Campylobacter jejuni strain HF5-...,/home/ubuntu/Disks/HDD1000/NCBI/20190704/refseq/,bacteria/GCF_001951255.1/GCF_001951255.1_ASM19...,2,Bacteria,194,Campylobacter,197,Campylobacter jejuni,1224,Proteobacteria,29547,Epsilonproteobacteria,72294,Campylobacteraceae,213849,Campylobacterales,0,NaN


In [168]:
selected_species = df[(df.type == "complete genome") 
   & (df.length > 1000000) 
   & ((df.name_superkingdom == "Bacteria") 
      | (df.name_superkingdom == "Archea"))].sample(100)[
    ["taxon_id", "taxon_name", "type", "length", "record_id", "path_folder", "path_file",
     "name_superkingdom", "record_description", "name_genus"]]

In [169]:
selected_species["path"] = selected_species.path_folder.str.cat(selected_species.path_file)
selected_species.drop(columns=["path_folder", "path_file"], inplace=True)
selected_species

,taxon_id,taxon_name,type,length,record_id,name_superkingdom,record_description,name_genus,path
8311,134601,Mycolicibacterium goodii,complete genome,7105933,NZ_CP012150.1,Bacteria,"NZ_CP012150.1 Mycobacterium goodii strain X7B,...",Mycolicibacterium,/home/ubuntu/Disks/HDD1000/NCBI/20190704/refse...
4602,1248726,Helicobacter pylori OK310,complete genome,1591278,NC_020509.1,Bacteria,"NC_020509.1 Helicobacter pylori OK310 DNA, com...",Campylobacterales,/home/ubuntu/Disks/HDD1000/NCBI/20190704/refse...
1323,329726,Acaryochloris marina MBIC11017,complete genome,6503724,NC_009925.1,Bacteria,"NC_009925.1 Acaryochloris marina MBIC11017, co...",NaN,/home/ubuntu/Disks/HDD1000/NCBI/20190704/refse...
11414,1340842,Campylobacter jejuni subsp. jejuni str. RM3420,complete genome,1666780,NZ_CP017456.1,Bacteria,NZ_CP017456.1 Campylobacter jejuni subsp. jeju...,Campylobacterales,/home/ubuntu/Disks/HDD1000/NCBI/20190704/refse...
9112,287,Pseudomonas aeruginosa,complete genome,6322459,NZ_CP008867.1,Bacteria,NZ_CP008867.1 Pseudomonas aeruginosa strain T5...,Pseudomonadaceae,/home/ubuntu/Disks/HDD1000/NCBI/20190704/refse...
...,...,...,...,...,...,...,...,...,...
8251,470,Acinetobacter baumannii,complete genome,3857743,NZ_CP012004.1,Bacteria,NZ_CP012004.1 Acinetobacter baumannii ATCC 179...,Pseudomonadales,/home/ubuntu/Disks/HDD1000/NCBI/20190704/refse...
1340,293614,Rickettsia akari str. Hartford,complete genome,1231060,NC_009881.1,Bacteria,"NC_009881.1 Rickettsia akari str. Hartford, co...",Alphaproteobacteria,/home/ubuntu/Disks/HDD1000/NCBI/20190704/refse...
9950,1827469,Brevundimonas sp. GW460-12-10-14-LB2,complete genome,3263775,NZ_CP015511.1,Bacteria,NZ_CP015511.1 Brevundimonas sp. GW460-12-10-14...,Brevundimonas sp. GW460-12-10-14-LB2,/home/ubuntu/Disks/HDD1000/NCBI/20190704/refse...
6133,1420012,Klebsiella pneumoniae 30660/NJST258_1,complete genome,5263229,NZ_CP006923.1,Bacteria,NZ_CP006923.1 Klebsiella pneumoniae 30660/NJST...,Enterobacterales,/home/ubuntu/Disks/HDD1000/NCBI/20190704/refse...


In [170]:
seeds = []
for row in tqdm(selected_species.itertuples()):
#     print(row.taxon_id)
    #  record_id, taxon, what, s_kingdom, folder, file    row["type"]
    #  record_id, taxon, genome, what, s_kingdom, path_complete, description
    sr = SyntRead(row.record_id, row.taxon_id, row.taxon_name, row.type, row.name_superkingdom, row.path, row.record_description)
    sr.synthetize_read()
    print(sr)
    seeds.append(sr)

Synthetic read from Mycolicibacterium goodii, len=1494, [6261127:6262621]
Synthetic read from Helicobacter pylori OK310, len=5111, [1009290:1014401]
Synthetic read from Acaryochloris marina MBIC11017, len=6231, [4472885:4479116]
Synthetic read from Campylobacter jejuni subsp. jejuni str. RM3420, len=5574, [549552:555126]
Synthetic read from Pseudomonas aeruginosa, len=1740, [5665363:5667103]
Synthetic read from Escherichia coli, len=6554, [1953601:1960155]
Synthetic read from Klebsiella quasipneumoniae, len=2826, [2548648:2551474]
Synthetic read from Campylobacter jejuni, len=9777, [1375625:1385402]
Synthetic read from Campylobacter jejuni subsp. jejuni, len=1578, [237963:239541]
Synthetic read from Chelatococcus sp. CO-6, len=2013, [3650172:3652185]
Synthetic read from Bordetella parapertussis 12822, len=8189, [1563349:1571538]
Synthetic read from Rivularia sp. PCC 7116, len=7821, [224936:232757]
Synthetic read from Francisella tularensis subsp. tularensis NE061598, len=4210, [148992:

In [172]:
synt_reads = []
for seed in tqdm(seeds):
    synt_reads.extend([seed] + seed.deep_random_copy(1000 -1))

In [173]:
path_out_syntreads = "/home/ubuntu/Data/Segmentation/Test-Data/Synthetic_from_Genomes/2019-11-22_SyntReads_100-BacArc.fastq"
SyntRead.to_fastq(synt_reads, path_out_syntreads)

# End

In [79]:
# df.info(memory_usage='deep')
f"{df.memory_usage(deep=True).sum()/1000000:.1f} MB"

'461.5 MB'

In [39]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 500)

In [76]:
df[(df["type"] == "undefined") 
   & ~(df.record_description.str.lower().str.contains("drosophila melanogaster")) 
   & ~(df.record_description.str.lower().str.contains("plasmid")) 
   & ~(df.record_description.str.lower().str.contains("scaffold")) 
   & ~(df.record_description.str.lower().str.contains("whole genome shotgun sequence")) 
   & ~(df.record_description.str.lower().str.contains("contig"))][["record_description", "length"]] 

,record_description,length
27,"NC_005791.1 Methanococcus maripaludis strain S2, complete sequence",1661137
236,NC_023044.1 Methanobacterium sp. MB1 complete sequence,2029766
249,NZ_CP007551.1 Haloferax mediterranei ATCC 33500 genome,2946877
257,NZ_CP008822.1 Metallosphaera sedula strain CuR1 genome,2191492
327,NZ_CP013695.1 Sulfolobus acidocaldarius strain NG05B_CO5_07 genome,2217426
356,NZ_CP019470.1 Methanopyrus sp. KOL6 genome,1430309
368,NZ_CP015193.1 Complete genome sequence of Thermococcus chitonophagus type strain GC74,1961979
379,NZ_CP017881.1 Methanohalophilus portucalensis strain FDF-1T genome,2084975
525,NC_002655.2 Escherichia coli O157:H7 str. EDL933 genome,5528445
595,"NC_002940.2 [Haemophilus] ducreyi 35000HP, complete sequence",1698955


In [80]:
df.shape

(360393, 25)

In [83]:
for col in df.columns:
    if "id_" in col:
        print(col)
        df[col].fillna(0, inplace=True)
        df[col] = df[col].astype(int)

id_superkingdom
id_kingdom
id_phylum
id_class
id_order
id_family
id_genus
id_species


'461.5 MB'

In [84]:
col_categories = ['rank', 'type', 'path_folder', 
                  'id_superkingdom', 'name_superkingdom', 'id_kingdom', 'name_kingdom', 'id_phylum', 'name_phylum', 
                  'id_class', 'name_class', 'id_order', 'name_order', 'id_family', 'name_family', 
                  'id_genus', 'name_genus', 'id_species', 'name_species']

for col in tqdm(col_categories):
    df[col] = df[col].astype('category')
print(f"{df.memory_usage(deep=True).sum()/1000000:.1f} MB")


179.7 MB


In [6]:
desired_ranks = ['superkingdom', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
headers = ["taxon_id", "taxon_name", "rank", "type", "length", 
           "record_id", "record_description", "path_folder", "path_file", ]
[headers.extend([f"id_{v}", f"name_{v}"]) for v in desired_ranks]
headers

['taxon_id',
 'taxon_name',
 'rank',
 'type',
 'length',
 'record_id',
 'record_description',
 'path_folder',
 'path_file',
 'id_superkingdom',
 'name_superkingdom',
 'id_kingdom',
 'name_kingdom',
 'id_phylum',
 'name_phylum',
 'id_class',
 'name_class',
 'id_order',
 'name_order',
 'id_family',
 'name_family',
 'id_genus',
 'name_genus',
 'id_species',
 'name_species']

# Read the csv and transform to pandas, plus column dtype conversion

Might take a while if file is big...

# End

In [144]:
path_file = "/home/ubuntu/Disks/HDD1000/NCBI/20190704/refseq/bacteria/GCF_000006845.1/GCF_000006845.1_ASM684v1_genomic.fna"

In [150]:
record = SeqIO.parse(path_file, "fasta")

In [151]:
r = next(record)
r

SeqRecord(seq=Seq('ATAAATTTTTGCACGGGTTGTGGATAAAATATCGGCGAGTCGGTATAATCGGTT...TGG', SingleLetterAlphabet()), id='NC_002946.2', name='NC_002946.2', description='NC_002946.2 Neisseria gonorrhoeae FA 1090 chromosome, complete genome', dbxrefs=[])

In [59]:
len(record.seq)

1936387

In [85]:
record.seq.upper()

Seq('AATTTAAAGATTAAAATTAGTAGACTGTCGATTTACAATATCATATTTATGAGT...TAA', SingleLetterAlphabet())

In [73]:
print(record)

ID: NC_014222.1
Name: NC_014222.1
Description: NC_014222.1 Methanococcus voltae A3, complete genome
Number of features: 0
Seq('AATTTAAAGATTAAAATTAGTAGACTGTCGATTTACAATATCATATTTATGAGT...TAA', SingleLetterAlphabet())


In [62]:
dir(record)

['__add__',
 '__bool__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__le___',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_per_letter_annotations',
 '_seq',
 '_set_per_letter_annotations',
 '_set_seq',
 'annotations',
 'dbxrefs',
 'description',
 'features',
 'format',
 'id',
 'letter_annotations',
 'lower',
 'name',
 'reverse_complement',
 'seq',
 'translate',
 'upper']

In [86]:
record.description

'NC_014222.1 Methanococcus voltae A3, complete genome'

In [176]:
get_desired_ranks(2, desired_ranks)

{'superkingdom_id': 2,
 'kingdom_id': 'NaN',
 'phylum_id': 'NaN',
 'class_id': 'NaN',
 'order_id': 'NaN',
 'family_id': 'NaN',
 'genus_id': 'NaN',
 'species_id': 'NaN'}

In [154]:
get_desired_ranks(9606, desired_ranks, tolist=True)

[2759, 33208, 7711, 40674, 9443, 9604, 9605, 9606]

In [155]:
ncbi.translate_to_names(get_desired_ranks(9606, desired_ranks, tolist=True))

['Eukaryota',
 'Metazoa',
 'Chordata',
 'Mammalia',
 'Primates',
 'Hominidae',
 'Homo',
 'Homo sapiens']

In [184]:
ranks_id_str = ncbi.get_taxid_translator(get_desired_ranks(6, desired_ranks, tolist=True))
ranks_id_str

{2: 'Bacteria',
 6: 'Azorhizobium',
 356: 'Rhizobiales',
 1224: 'Proteobacteria',
 28211: 'Alphaproteobacteria',
 335928: 'Xanthobacteraceae'}

In [185]:
l=[]
[l.extend([k,v]) for k,v in ranks_id_str.items()]

[None, None, None, None, None, None]

In [186]:
l

[2,
 'Bacteria',
 6,
 'Azorhizobium',
 356,
 'Rhizobiales',
 1224,
 'Proteobacteria',
 28211,
 'Alphaproteobacteria',
 335928,
 'Xanthobacteraceae']